In [1]:
import os
import datetime
import cv2
import glob
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
import keras

import keras_tuner as kt


2023-01-09 14:02:20.727525: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 14:02:20.829020: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-09 14:02:20.852142: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-09 14:02:21.276845: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

# Get data

data is already augmented

In [2]:
data_dir = os.path.join("..", "yolo_preperation", "data")

In [3]:
def import_yolo_data(data_dir, split, stride=1):
    image_path = os.path.join(data_dir, "images", split)
    label_path = os.path.join(data_dir, "labels", split)
    
    images = []
    labels = []
    
    i = 0
    
    for file_name in glob.glob(os.path.join(image_path, "*.jpg")):
        if i % stride == 0:
            img = cv2.imread(file_name)
            height, width = img.shape[:2]
            images.append(cv2.resize(img, (int(width / 2), int(height / 2))) / 255.0)
        i += 1
        
        
    i = 0
    
    for label_txt in glob.glob(os.path.join(label_path, "*.txt")):
        if i % stride == 0:
            with open(label_txt) as f:
                labels.append(int(f.readlines()[0].split(" ")[0]))
        i += 1
            
    return (images, labels)

In [4]:
stride = 16
train_X, train_y = import_yolo_data(data_dir, "train", stride)
val_X, val_y = import_yolo_data(data_dir, "val", stride)
test_X, test_y = import_yolo_data(data_dir, "test", stride)

In [5]:
# train_X = np.array(train_X).flatten().reshape(len(train_X), -1)
# val_X = np.array(val_X).flatten().reshape(len(val_X), -1)
# test_X = np.array(test_X).flatten().reshape(len(test_X), -1)

In [6]:
train_X = np.array(train_X)
val_X = np.array(val_X)
test_X = np.array(test_X)

In [7]:
train_y = OneHotEncoder().fit_transform(np.array(train_y).reshape(-1,1)).reshape(-1, 4).toarray()
val_y = OneHotEncoder().fit_transform(np.array(val_y).reshape(-1,1)).reshape(-1, 4).toarray()
test_y = OneHotEncoder().fit_transform(np.array(test_y).reshape(-1,1)).reshape(-1, 4).toarray()

class_names = ["0", "1", "2", "3"]

In [8]:
print(train_X[0].shape)
print(train_y[0].shape)

(360, 640, 3)
(4,)


In [9]:
print(len(train_X))
print(len(train_y))
print()
print(len(val_X))
print(len(val_y))
print()
print(len(test_X))
print(len(test_y))

300
300

100
100

100
100


In [18]:
def model_builder(hp):
    model = Sequential([
    keras.layers.Conv2D(32, (3, 3), input_shape = (360, 640, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation = 'relu', use_bias=True, bias_initializer='ones'),
    keras.layers.Dense(4, activation = 'softmax', use_bias=True, bias_initializer='ones'),
    ])
    hp_learning_rate = hp.Choice('learning_rate', values = [0.5, 1e-1, 1e-2, 1e-3, 1e-4])
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate), metrics=['accuracy'])
    return model

In [26]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'kt_cnn',
                     project_name = 'cnnv2_1')

In [27]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir)     

In [28]:
tuner.search(train_X, train_y, epochs = 50,
             validation_data = (val_X, val_y),
             callbacks=[tensorboard_callback])

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.30000001192092896

Best val_accuracy So Far: 0.30000001192092896
Total elapsed time: 00h 00m 31s
INFO:tensorflow:Oracle triggered exit


In [29]:
best_hps = tuner.get_best_hyperparameters()[0]
print(f"""
The hyperparameter search is complete. The optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal learning rate for the optimizer
is 0.0001.



In [30]:
tuner.search_space_summary()

best_model = tuner.get_best_models()[0]
best_model.summary()
best_model.evaluate(val_X, val_y)

Search space summary
Default search space size: 1
learning_rate (Choice)
{'default': 0.5, 'conditions': [], 'values': [0.5, 0.1, 0.01, 0.001, 0.0001], 'ordered': True}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 358, 638, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 179, 319, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 177, 317, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 88, 158, 32)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)  

[1.385679006576538, 0.30000001192092896]

In [31]:
tuner.results_summary()

Results summary
Results in kt_cnn/cnnv2_1
Showing 10 best trials
Trial summary
Hyperparameters:
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.30000001192092896
Trial summary
Hyperparameters:
learning_rate: 0.5
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.30000001192092896
Trial summary
Hyperparameters:
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.30000001192092896
Trial summary
Hyperparameters:
learning_rate: 0.1
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.30000001192092896
Trial summary
Hyperparameters:
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.28999999165534973
